### Afficher un échantillon d’images pour chaque classe avec les boîtes englobantes.

In [ ]:
image1_path = '../data/raw/Data_Brain/Train/Glioma/images/gg (1).jpg'
image2_path = '../data/raw/Data_Brain/Train/Meningioma/images/m (2).jpg'
image3_path = '../data/raw/Data_Brain/Train/No Tumor/images/1.jpg'
image4_path = '../data/raw/Data_Brain/Train/Pituitary/images/p (1).jpg'

import cv2
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))

# Glioma

image = cv2.imread(image1_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
x_min, y_min, x_max, y_max = 200, 200, 350, 350
cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

plt.subplot(2, 2, 1)
plt.imshow(image_rgb)
plt.title("Glioma")
plt.axis('off')

# Meningioma

image = cv2.imread(image2_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
x_min, y_min, x_max, y_max = 230, 160, 350, 260
cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

plt.subplot(2, 2, 2)
plt.imshow(image_rgb)
plt.title("Meningioma")
plt.axis('off')

# No Tumor

image = cv2.imread(image3_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.subplot(2, 2, 3)
plt.imshow(image_rgb)
plt.title("No Tumor")
plt.axis('off')

# Petuitary

image = cv2.imread(image4_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
x_min, y_min, x_max, y_max = 200, 160, 320, 260
cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

plt.subplot(2, 2, 4)
plt.imshow(image_rgb)
plt.title("Petuitary")
plt.axis('off')



plt.show()



### Créer un dossier pour stocker les données après filtrage.

In [49]:
import os

dir_path = "../data/processed/Data_Brain"

try:
    os.mkdir(dir_path)
except:
    print("Directory already exists")

Directory already exists


### Pour chaque image : Vérifier si un fichier .txt correspondant existe.

- Si oui : copier l’image et le label vers les dossiers outputpath/images/ et outputpath/labels/ (train, valid, test).

- Si non : afficher un message d’avertissement et ignorer l’image.

In [ ]:
raw_data_path = "../data/raw/Data_Brain"

os.listdir()